project workflow:
1. config.yaml
2. params.yaml
3. contants
4. entity
5. component
6. pipeline

In [1]:
!cd

e:\SummerProgram\Projects\Image_Bases_DL_Projects\End_To_End_Chicken_Disease_Classifier\research


In [2]:
import os 
os.chdir("../")

In [3]:
!cd

e:\SummerProgram\Projects\Image_Bases_DL_Projects\End_To_End_Chicken_Disease_Classifier


#### 1.config/config.yaml code

artifact_dir : "artifacts"

data_ingestion:

        root_dir : "artifacts/data_ingestion" 

        souce_URL : "https://github.com/arosha27/Datasets/data.zip"

        local_data_file : "artifacts/data_ingestion/data.zip"

        unzip_dir : "artifacts/data_ingestion"



#### 2. src/cnnClassifier/constant/__init__.py

from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")

PARAMS_FILE_PATH = Path("params.yaml")

### 3. src/cnnClassifier/entity/configuration_entity.py

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen = True)

class DataIngestionConfig:
    root_dir : Path 
    source_URL : str
    local_data_file : Path
    unzip_dir : Path
    
    

#### 4. src/cnnClassifier/config/configuration.py


In [5]:
from cnnClassifier.constants import*
from cnnClassifier.utils.common import read_yaml, create_directories



In [6]:
class ConfigurationManeger:
    def __init__(self , 
                 config_fiepath = CONFIG_FILE_PATH ,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_fiepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir   
            
        )
        
        return data_ingestion_config
        
        

#### 5. src/cnnClassifier/component/01_data_ingestion.py

In [7]:
import os 
import zipfile
import urllib.request as request
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self , config : DataIngestionConfig):
        self.config = config
       
       #Dowload the data 
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(url = self.config.source_URL, filename = self.config.local_data_file)
            logger.info(f"{filename} downloaded with following info : \n{headers}")
            
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")
        #Unzip the zip file    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file , "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        
            

#### 6. src/cnnClassifier/pipeline/stage_01_data_ingestion.py

In [10]:
try:
    config = ConfigurationManeger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e
    


[2025-11-08 23:59:30,922: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-08 23:59:30,925: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-08 23:59:30,928: INFO: common: created directory at: artifacts]
[2025-11-08 23:59:30,929: INFO: common: created directory at: artifacts/data_ingestion]
[2025-11-09 00:00:03,980: INFO: 302602151: artifacts/data_ingestion/data.zip downloaded with following info : 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 48E0:256ED2:F223CC:121A502:690F9302
Accept-Ranges: bytes
Date: Sat, 08 Nov 2025 18:59:16 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990021-FJR
X-Cach